<a href="https://www.kaggle.com/code/georgelukaanya/bee-catcher-rv?scriptVersionId=243695666" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# ====================
# Dependencies
# ====================
!pip install roboflow ultralytics

import os
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
# ====================
# 1. Download Roboflow Dataset
# ====================
rf = Roboflow(api_key="QpIHmdiUrfY49fQ9rsgu")
project = rf.workspace("rodri-6xa7t").project("bee-labels-zkh7p")
version = project.version(2)

# This returns a Roboflow Dataset object
dataset = version.download("yolov8")

# Extract the local folder path (string) from the Dataset object
dataset_dir = dataset.location  # e.g. "./Bee-labels-2"

In [ ]:
# ====================
# 2. (Optional) Inspect data.yaml
# ====================
# with open(os.path.join(dataset_dir, 'data.yaml')) as f:
#     print(f.read())

# ====================
# 3. Initialize YOLOv8 Model
# ====================
# You can choose any pre‑trained YOLOv8 variant or your own weights
model = YOLO('yolov8s.pt')

# ====================
# 4. Train
# ====================
results = model.train(
    data=os.path.join(dataset_dir, 'data.yaml'),
    epochs=100,
    imgsz=640,
    batch=16,
    project='rf_bee_detection',
    name='yolov8_bees',
    exist_ok=True,      # overwrite existing runs of the same name
    # augmentations & hyperparameters (optional)
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    patience=20
)

# ====================
# 5. Validate
# ====================
metrics = model.val(
    data=os.path.join(dataset_dir, 'data.yaml'),
    conf=0.25,
    iou=0.5
)
print(metrics)

In [ ]:
# ====================
# 6. Inference on New Images
# ====================
preds = model.predict(
    source='/kaggle/working/Bee-labels-2/test/images/',  # adjust this path
    conf=0.25,
    save=True,      # saves annotated images
    save_txt=True   # saves YOLO-format .txt predictions
)

# ====================
# 7. (Optional) Export to Other Formats
# ====================
# e.g. TensorFlow Lite
model.export(format='tflite')
